# Analysis of Belle2 Data

In [1]:
import os, sys, pprint
import glob
from root_pandas import read_root
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

Welcome to JupyROOT 6.16/00


/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.4 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [2]:
!ls -l ../DATA/
!pwd

total 56352
-rwxr--r--  1 philipp philipp   584128 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_ccbar_0.root
-rwxr--r--  1 philipp philipp   117621 May 14 14:58 B0_etapr-eta-gg2pi_KS-pi+pi-_output_charged_17.root
-rwxr--r--  1 philipp philipp   155084 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_0.root
-rwxr--r--  1 philipp philipp   177149 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_1.root
-rwxr--r--  1 philipp philipp   183977 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_2.root
-rwxr--r--  1 philipp philipp   176779 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_3.root
-rwxr--r--  1 philipp philipp   185028 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_4.root
-rwxr--r--  1 philipp philipp   141664 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_5.root
-rwxr--r--  1 philipp philipp   192110 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_6.root
-rwxr--r--  1 philipp philipp   179731 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_7.root

## Reading and Prepairing Data

In [3]:
path = '/home/philipp/Desktop/Project/DATA/'
SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
CFiles = glob.glob(os.path.join(path, 'Continuous/*.root'))
PFiles = glob.glob(os.path.join(path, 'Peaking/*.root'))

Full_Signal = pd.concat((read_root(f, 'B0') for f in SFiles))
Full_Continuous = pd.concat((read_root(f, 'B0') for f in CFiles))
Full_Peaking = pd.concat((read_root(f, 'B0') for f in PFiles))

/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/root_pandas/readwrite.py:320: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(columns, axis=1, copy=False)


In [4]:
Full_Continuous

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
0,0,0,51571101,0.500895,0.002853,1.149850,5.135701,0.012654,-11.366517,0.938931,...,"[0.0019425296, -0.0031857349, 0.005766661]",-111.0,-111.0,-111.0,0,14707.006836,0.261846,-1.036396,1,0
1,0,0,51572506,0.496650,0.002876,-0.335105,5.268522,0.019502,-0.564453,0.916602,...,"[-0.0017205713, 0.0036101926, 0.020706093]",-111.0,-111.0,-111.0,0,14707.006836,0.134443,-1.862235,2,0
2,0,0,51572506,0.496650,0.002876,-0.335105,5.184201,0.021533,-4.427056,0.924155,...,"[-0.0017205713, 0.0036101926, 0.020706093]",-111.0,-111.0,-111.0,0,14707.006836,0.096710,-2.234329,2,1
3,0,0,52815187,0.512523,0.003407,4.376027,5.264990,0.018447,-0.788202,0.952054,...,"[-0.0011433143, 1.0134625e-06, -0.00859407]",-111.0,-111.0,-111.0,0,14707.006836,0.155727,-1.690369,1,0
4,0,0,52818414,0.499365,0.003113,0.562306,5.200452,0.021283,-3.715473,0.939792,...,"[0.00070354354, -3.8584048e-07, -0.006386356]",-111.0,-111.0,-111.0,0,14707.006836,0.009158,-4.683872,1,0
5,0,0,52438085,0.507719,0.004798,2.106009,5.424548,0.042539,3.409029,0.922027,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.003701,-5.595561,4,0
6,0,0,52438085,0.507719,0.004798,2.106009,5.441957,0.042845,3.791087,0.913086,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.003192,-5.743785,4,1
7,0,0,52438085,0.515977,0.005975,3.073336,5.410072,0.032874,3.970957,0.922027,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.006637,-5.008420,4,2
8,0,0,52438085,0.515977,0.005975,3.073336,5.427326,0.033301,4.438130,0.913086,...,"[7.33112e-05, -2.2381237e-06, 0.016870165]",-111.0,-111.0,-111.0,0,14707.006836,0.007339,-4.907236,4,3
9,0,0,117041010,0.506843,0.033634,0.274386,5.316215,0.124951,0.293591,0.949488,...,"[0.0, 0.0, 0.0]",-111.0,-111.0,-111.0,0,14707.006836,0.515877,0.063531,1,0


__Branches, organized by Categories__

In [11]:
General = ['exp_no', 'run_no', 'evt_no', 'nCands', 'iCand']
Masses = ['B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 
          'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM']
Kinetics = ['B0_P', 'B0_P4', 'B0_deltae', 'B0_mbc', 'B0_etap_P', 'B0_etap_P4', 
            'B0_etap_eta_P', 'B0_etap_eta_P4', 'B0_etap_eta_gamma0_P', 'B0_etap_eta_gamma0_P4',
            'B0_etap_eta_gamma1_P', 'B0_etap_eta_gamma1_P4', 'B0_etap_pi0_P', 'B0_etap_pi0_P4',
            'B0_etap_pi1_P', 'B0_etap_pi1_P4', 'B0_K_S0_P', 'B0_K_S0_P4']
Kinetics_comp = ['B0_P', 'B0_P40', 'B0_P41', 'B0_P42', 'B0_P43', 'B0_deltae', 'B0_mbc',
                 'B0_etap_P', 'B0_etap_P40', 'B0_etap_P41', 'B0_etap_P42','B0_etap_P43','B0_etap_eta_P',
                 'B0_etap_eta_P40', 'B0_etap_eta_P41', 'B0_etap_eta_P42', 'B0_etap_eta_P43',
                 'B0_etap_eta_gamma0_P', 'B0_etap_eta_gamma0_P40', 'B0_etap_eta_gamma0_P41', 
                 'B0_etap_eta_gamma0_P42', 'B0_etap_eta_gamma0_P43', 'B0_etap_eta_gamma1_P', 'B0_etap_eta_gamma1_P40',
                 'B0_etap_eta_gamma1_P41', 'B0_etap_eta_gamma1_P42', 'B0_etap_eta_gamma1_P43', 'B0_etap_pi0_P',
                 'B0_etap_pi0_P40', 'B0_etap_pi0_P41', 'B0_etap_pi0_P42', 'B0_etap_pi0_P43', 'B0_etap_pi1_P',
                 'B0_etap_pi1_P40', 'B0_etap_pi1_P41', 'B0_etap_pi1_P42', 'B0_etap_pi1_P43', 'B0_K_S0_P',
                 'B0_K_S0_P40', 'B0_K_S0_P41', 'B0_K_S0_P42', 'B0_K_S0_P43']
FourMomenta = ['B0_P4', 'B0_etap_P4', 'B0_etap_eta_P4', 'B0_etap_eta_gamma0_P4', 'B0_etap_eta_gamma1_P4', 
            'B0_etap_pi0_P4', 'B0_etap_pi1_P4', 'B0_K_S0_P4']
DecayAngles = ['B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc',
               'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc']
Positions = ['B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho',  
             'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 
             'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho',
             'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y',
             'B0_etap_eta_ErrY', 'B0_etap_eta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho',
             'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 
             'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 
             'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 
             'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 
             'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z',
             'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 
             'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance',
             'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr']
Vertex = ['B0_VtxPvalue', 'B0_VtxProd', 'B0_VtxProdCov', 'B0_etap_VtxPvalue', 'B0_etap_VtxProd', 'B0_etap_VtxProdCov',
          'B0_etap_eta_VtxPvalue', 'B0_etap_eta_VtxProd', 'B0_etap_VtxProdCov', 
          'B0_etap_pi0_VtxPvalue', 'B0_etap_pi0_VtxProd', 'B0_etap_pi0_VtxProdCov', 
          'B0_etap_pi1_VtxPvalue', 'B0_etap_pi1_VtxProd', 'B0_etap_pi1_VtxProdCov', 
          'B0_K_S0_VtxPvalue', 'B0_K_S0_VtxProd', 'B0_K_S0_VtxProdCov']
Continuum_Supression = ['B0_CSMVA', 'B0_TrCSMVA']
Training = Masses + Kinetics + DecayAngles + Positions + Vertex + Continuum_Supression

In [6]:
Training

['B0_M',
 'B0_ErrM',
 'B0_SigM',
 'B0_K_S0_M',
 'B0_K_S0_ErrM',
 'B0_K_S0_SigM',
 'B0_etap_M',
 'B0_etap_ErrM',
 'B0_etap_SigM',
 'B0_etap_eta_M',
 'B0_etap_eta_ErrM',
 'B0_etap_eta_SigM',
 'B0_P',
 'B0_P40',
 'B0_P41',
 'B0_P42',
 'B0_P43',
 'B0_deltae',
 'B0_mbc',
 'B0_etap_P',
 'B0_etap_P40',
 'B0_etap_P41',
 'B0_etap_P42',
 'B0_etap_P43',
 'B0_etap_eta_P',
 'B0_etap_eta_P40',
 'B0_etap_eta_P41',
 'B0_etap_eta_P42',
 'B0_etap_eta_P43',
 'B0_etap_eta_gamma0_P',
 'B0_etap_eta_gamma0_P40',
 'B0_etap_eta_gamma0_P41',
 'B0_etap_eta_gamma0_P42',
 'B0_etap_eta_gamma0_P43',
 'B0_etap_eta_gamma1_P',
 'B0_etap_eta_gamma1_P40',
 'B0_etap_eta_gamma1_P41',
 'B0_etap_eta_gamma1_P42',
 'B0_etap_eta_gamma1_P43',
 'B0_etap_pi0_P',
 'B0_etap_pi0_P40',
 'B0_etap_pi0_P41',
 'B0_etap_pi0_P42',
 'B0_etap_pi0_P43',
 'B0_etap_pi1_P',
 'B0_etap_pi1_P40',
 'B0_etap_pi1_P41',
 'B0_etap_pi1_P42',
 'B0_etap_pi1_P43',
 'B0_K_S0_P',
 'B0_K_S0_P40',
 'B0_K_S0_P41',
 'B0_K_S0_P42',
 'B0_K_S0_P43',
 'B0_decayAngle__

#### Choosing the best candidates in each event

Combinatorics in the reconstruction process often leads to events with multiple candidates, where we carefully have to choose the best without introducing significant bias. To do this we have two possible methods: 

- Choosing the candidate with the reconstructed mass closest to the nominal value
- Choosing the candidate with the best vertex fit

If we were only using the mass we would bias our data towards a very narrow mass peak in cases where two candidates are close to the nominal value. Therefore I suggest using the masses for a generous cut (e.g. $5\sigma$) and then using vertex information to pick the best candidate. 

##### Masses
Nominal masses with errors in GeV $ [M_{nominal}, \sigma_{M_{nominal}}] $ according to the [Particle Data Group](http://pdg.lbl.gov/2019/listings/contents_listings.html)

In events with more than one candidate we can use this to perform a rough cut on candidates requirering, that 

$$ 
M_{reconstructed} \in 
    \left[ M_{nominal} - 5 \times \left( \sigma_{M_{nominal}} + \sigma_{M_{reconstructed}} \right), 
           M_{nominal} + 5 \times \left( \sigma_{M_{nominal}} + \sigma_{M_{reconstructed}} \right) \right]
$$



In [8]:
M_B0 = [5.27955, 0.00026]
M_K0 = [0.497611, 0.000013]
M_etap = [0.95778, 0.00006]
M_eta = [0.547862, 0.000017]

In [9]:
#Signal = Signal[abs(Signal['B0_M'] - M_B0[0]) < 5 * Signal['B0_ErrM']]
#Signal = Signal[abs(Signal['B0_K_S0_M'] - M_K0[0]) < 5 * Signal['B0_K_S0_ErrM']]
#Signal = Signal[abs(Signal['B0_etap_M'] - M_etap[0]) < 5 * Signal['B0_etap_ErrM']]
#Signal = Signal[abs(Signal['B0_etap_eta_M'] - M_eta[0]) < 5 * Signal['B0_etap_eta_ErrM']]

##### Vertex

I need to take a closer look at the vertex information to decide how to use it

In [ ]:
# Do stuff

#### Filtering the Signal and chosing variables to train
- We filter the Signal, as we are only interested in correctly reconstructed events
- We choose variables for the training
- We convert arrays to 4 variables

In [12]:
Pure_Signal = Full_Signal[Full_Signal['B0_isSignal']==1].reset_index(drop=True)

Train_Signal = Pure_Signal[Training]
Train_Continuous = Full_Continuous[Training]
Train_Peaking = Full_Peaking[Training]

#Signal_comp = pd.concat([pd.DataFrame(Signal_Pure[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Signal_Pure[c] for c in Signal_Pure], axis=1)
#Background_comp = pd.concat([pd.DataFrame(Background[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Background[c] for c in Background], axis=1)

In [13]:
for M in FourMomenta: 
    Train_Signal[M+'0'], Train_Signal[M+'1'], Train_Signal[M+'2'], Train_Signal[M+'3'] = zip(*Train_Signal.pop(M))

/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
Train_Signal[Kinetics_comp]

,B0_P,B0_P40,B0_P41,B0_P42,B0_P43,B0_deltae,B0_mbc,B0_etap_P,B0_etap_P40,B0_etap_P41,...,B0_etap_pi1_P,B0_etap_pi1_P40,B0_etap_pi1_P41,B0_etap_pi1_P42,B0_etap_pi1_P43,B0_K_S0_P,B0_K_S0_P40,B0_K_S0_P41,B0_K_S0_P42,B0_K_S0_P43
0,1.602125,0.314331,0.325830,1.536826,5.543979,0.028394,5.278986,3.066705,1.227109,2.410350,...,0.334137,0.075672,0.313344,0.087958,0.362115,2.277444,-0.912778,-2.084520,0.091455,2.331229
1,1.770375,0.015410,-0.344991,1.736367,5.520761,-0.039430,5.268792,1.871150,-1.096115,0.942214,...,0.615245,-0.273122,0.314300,-0.452930,0.630877,3.383169,1.111526,-1.287206,2.924628,3.418530
2,1.462492,0.394294,-0.313335,1.373039,5.467072,-0.008629,5.276476,2.782038,0.691355,2.128145,...,0.583644,0.299509,0.355504,0.352920,0.600100,2.475380,-0.297061,-2.441480,-0.280068,2.524832
3,1.514491,0.206613,-0.351200,1.458647,5.456528,-0.035768,5.277987,2.556790,-0.904877,2.073730,...,0.199408,-0.001667,0.189643,0.061614,0.243399,2.680938,1.111489,-2.424929,0.267841,2.725981
4,1.509634,-0.042359,-0.251473,1.487939,5.513276,0.025709,5.276795,2.603838,-1.631517,-2.017514,...,0.359964,-0.294196,-0.206384,-0.020684,0.386075,2.693635,1.589158,1.766041,1.269388,2.738285
5,1.691027,0.073549,0.222069,1.674768,5.577186,0.034224,5.280358,2.578136,0.832161,2.434409,...,0.409031,0.096112,0.378459,-0.121808,0.432187,2.782580,-0.758612,-2.212340,1.507585,2.826931
6,1.476038,-0.056365,-0.098082,1.471697,5.478603,-0.005152,5.281182,2.305398,-2.207834,-0.431153,...,0.545151,-0.469216,-0.250901,-0.118637,0.562734,2.940199,2.151469,0.333071,1.976112,2.982189
7,1.222074,0.222588,0.163780,1.190418,5.423142,0.004857,5.278788,2.703137,0.199800,-2.338269,...,0.611039,0.010155,-0.560089,0.244061,0.626776,2.506708,0.022789,2.502049,-0.151047,2.555246
8,1.776680,0.162885,-0.280004,1.746899,5.578936,0.011739,5.276704,2.808167,-2.363503,0.087459,...,0.823619,-0.774434,0.023507,0.279373,0.835362,2.563578,2.526387,-0.367463,0.232961,2.611701
9,1.560526,0.307232,-0.325742,1.494905,5.491690,-0.013811,5.279142,2.559568,-0.526277,-2.502656,...,0.462452,-0.138050,-0.431627,0.092207,0.483054,2.713678,0.833509,2.176914,1.389373,2.758825
